Un RDD (Resilient Distributed Dataset) es la estructura de datos básica de Apache Spark:
Es una colección inmutable de objetos distribuidos entre nodos de un clúster.

Sin embargo, los RDD no tienen esquema, es decir, Spark no sabe qué significan sus campos. Para poder usar consultas SQL o funciones de alto nivel, se convierten en DataFrame.

## Impotar librerías

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

Crear SparkSession (punto de entrada para Spark SQL)

In [14]:
spark = SparkSession.builder \
    .appName("Ejemplos Spark SQL") \
    .getOrCreate()

Cargar datos externos como DataFrame (JSON)

Para el JSON: Spark no acepta un bloque único [...] por defecto, sino un objeto por línea.

In [12]:
print("\n=== Ejemplo 1: Cargar JSON como DataFrame ===")
df = spark.read.json("people.json")
df.show()


=== Ejemplo 1: Cargar JSON como DataFrame ===
+---+-------+
|age|   name|
+---+-------+
| 29|  Alice|
| 35|    Bob|
| 40|Charlie|
| 21| Sergio|
| 80| Camilo|
|  9|Natalia|
| 20|  Diego|
+---+-------+



Registrar DataFrame como Vista Temporal para consultas SQL y
Consultas equivalentes con API de DataFrame

In [10]:
print("\n=== Ejemplo 2: Consultar con SQL ===")
df.createOrReplaceTempView("people")
result = spark.sql("SELECT name, age FROM people WHERE age > 30")
result.show()

print("\n=== Ejemplo 3: Consultar con API DataFrame ===")
df.select("name", "age").where(col("age") > 30).show()


=== Ejemplo 2: Consultar con SQL ===
+-------+---+
|   name|age|
+-------+---+
|    Bob| 35|
|Charlie| 40|
| Camilo| 80|
+-------+---+


=== Ejemplo 3: Consultar con API DataFrame ===
+-------+---+
|   name|age|
+-------+---+
|    Bob| 35|
|Charlie| 40|
| Camilo| 80|
+-------+---+



Operaciones de Agregación Básicas

In [15]:
print("\n=== Ejemplo 4: Operaciones de Agregación ===")
# Con SQL
spark.sql("""
    SELECT
        COUNT(*) as total_personas,
        AVG(age) as edad_promedio,
        MAX(age) as edad_maxima,
        MIN(age) as edad_minima
    FROM people
""").show()

# Con DataFrame API
from pyspark.sql.functions import count, avg, max, min
df.agg(
    count("*").alias("total_personas"),
    avg("age").alias("edad_promedio"),
    max("age").alias("edad_maxima"),
    min("age").alias("edad_minima")
).show()


=== Ejemplo 4: Operaciones de Agregación ===
+--------------+-----------------+-----------+-----------+
|total_personas|    edad_promedio|edad_maxima|edad_minima|
+--------------+-----------------+-----------+-----------+
|             7|33.42857142857143|         80|          9|
+--------------+-----------------+-----------+-----------+

+--------------+-----------------+-----------+-----------+
|total_personas|    edad_promedio|edad_maxima|edad_minima|
+--------------+-----------------+-----------+-----------+
|             7|33.42857142857143|         80|          9|
+--------------+-----------------+-----------+-----------+



Agrupación y Filtrado

In [16]:
print("\n=== Ejemplo 5: Agrupación por Rangos de Edad ===")
# Con SQL
spark.sql("""
    SELECT
        CASE
            WHEN age < 18 THEN 'Menor de edad'
            WHEN age BETWEEN 18 AND 30 THEN 'Joven'
            WHEN age BETWEEN 31 AND 50 THEN 'Adulto'
            ELSE 'Mayor'
        END as grupo_edad,
        COUNT(*) as cantidad,
        AVG(age) as edad_promedio
    FROM people
    GROUP BY grupo_edad
    ORDER BY cantidad DESC
""").show()

# Con DataFrame API
from pyspark.sql.functions import when
df.withColumn("grupo_edad",
    when(col("age") < 18, "Menor de edad")
    .when((col("age") >= 18) & (col("age") <= 30), "Joven")
    .when((col("age") > 30) & (col("age") <= 50), "Adulto")
    .otherwise("Mayor")
).groupBy("grupo_edad").agg(
    count("*").alias("cantidad"),
    avg("age").alias("edad_promedio")
).orderBy(col("cantidad").desc()).show()


=== Ejemplo 5: Agrupación por Rangos de Edad ===
+-------------+--------+------------------+
|   grupo_edad|cantidad|     edad_promedio|
+-------------+--------+------------------+
|        Joven|       3|23.333333333333332|
|       Adulto|       2|              37.5|
|        Mayor|       1|              80.0|
|Menor de edad|       1|               9.0|
+-------------+--------+------------------+

+-------------+--------+------------------+
|   grupo_edad|cantidad|     edad_promedio|
+-------------+--------+------------------+
|        Joven|       3|23.333333333333332|
|       Adulto|       2|              37.5|
|        Mayor|       1|              80.0|
|Menor de edad|       1|               9.0|
+-------------+--------+------------------+

